<a href="https://colab.research.google.com/github/step-cheng/CIFAR10-pytorch/blob/main/CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# !pip install torch>=2.0.1
# !pip install torchvision
!pip install -Uqq ipdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.0 MB/s eta 0:00:00


In [1]:
!git clone https://github.com/step-cheng/CIFAR10-pytorch.git


Cloning into 'CIFAR10-pytorch'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 43 (delta 15), reused 21 (delta 4), pack-reused 0
Receiving objects: 100% (43/43), 19.27 KiB | 6.42 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [3]:
# Turn on Google Colab Debugger
%pdb on
%pdb off

In [5]:
import torch
from torch import nn
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import ipdb

In [3]:
import sys
sys.path.insert(0,'/content/CIFAR10-pytorch')
from models import *

In [4]:
# 50000 train images
train_data = datasets.CIFAR10(root='./', train=True, download=True, transform=transforms.ToTensor(),
                              target_transform=transforms.Lambda(lambda y : torch.zeros(10,dtype=torch.float).scatter(dim=0,
                                                                                                                       index=torch.tensor(y),
                                                                                                                       value=1)))
# DataLoader wraps an iterable over dataset for automatic batching, sampling, and dataloading
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)

# 10000 test images
test_data = datasets.CIFAR10(root='./', train=False, download=True, transform=transforms.ToTensor(),
                             target_transform=transforms.Lambda(lambda y : torch.zeros(10,dtype=torch.float).scatter(dim=0,
                                                                                                                       index=torch.tensor(y),
                                                                                                                       value=1)))
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True)

classes = {
    0 : 'plane',    1 : 'car',    2 : 'bird',   3 : 'cat',    4 : 'deer',
    5 : 'dog',      6 : 'frog',   7 : 'horse',  8 : 'ship',   9 : 'truck'
}

100%|██████████| 170498071/170498071 [00:05<00:00, 29577102.35it/s]


Extracting ./cifar-10-python.tar.gz to ./
Files already downloaded and verified


In [5]:
# img, label = training_data[1]
train_iter = iter(train_loader)
# img, label = next(train_iter)
# print(img.shape)
# print(label.shape)

# shows image tensor
def show(tens, label):
  # make_grid makes a grid of images from a batch of tensors BxCxHxW
  label = torch.argmax(label,dim=1)
  print(label)
  img = torchvision.utils.make_grid(tens) # flattens B dimension to make a grid
  np_img = img.numpy()
  plt.imshow(np.transpose(np_img, (1,2,0)))     # seems each image is spaced by 2 pixels, channels goes last when plotting
  # title = classes[label]
  title = ' '.join([classes[t.item()] for t in label])
  plt.title(title)

# show(img, label)


In [ ]:
# # Demonstration of loss.backward() and optimizer.step()

# print(model.conv_pool_stack[0].weight[0,0])
# loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(),lr=0.1)

# input = torch.randn((4,3,32,32),requires_grad=False)
# target = torch.randint(0,10,size=(4,))
# logits = model(input)
# loss = loss_fn(logits,target)
# print(loss)
# loss.backward()
# print(model.conv_pool_stack[0].weight[0,0])
# print(model.conv_pool_stack[0].weight.grad[0,0])
# optimizer.step()
# print(model.conv_pool_stack[0].weight[0,0])
# print(model.conv_pool_stack[0].weight.grad[0,0])
# optimizer.zero_grad()

In [10]:
def preprocess(x):
  # find std and mean per channel
  x_stds = torch.std(x, dim=(0,2,3))
  x_means = torch.sum(x, dim=(0,2,3)) / (x.shape[0]*x.shape[2]*x.shape[3])

  for channel in range(3):
    x[:,channel,:,:] -= x_means[channel]
    x[:,channel,:,:] /= x_stds[channel]
  return x


tensor([1.0183, 1.0052, 0.9976])
tensor([1.0183, 1.0052, 0.9976])


In [7]:
def accuracy(pred,y):
  guesses = torch.argmax(pred,dim=1)
  matches = sum(guesses==y).item()
  return matches/torch.numel(y), matches


In [8]:
def train(dataloader, model, device, loss_fn, optimizer):
  # sets the module to training mode, eval does the opposite
  model.train()

  accs = []
  matches = 0

  for batch, (X, y) in enumerate(dataloader):
    imgs, labels = X.to(device), y.to(device)

    # do data preprocessing
    imgs = preprocess(imgs)
    # do not call model.foward(X), doing model(X) calls the __call__() function which does a few extra hooks...
    pred = model(imgs)
    loss = loss_fn(pred, torch.argmax(labels,dim=1))
    acc, m = accuracy(pred,torch.argmax(labels,dim=1))
    matches += m

    if batch % 50 == 0:
      accs.append(acc)
      print(f"Progress: {(batch+1) * len(X)}, current accuracy: {m/len(X)}")

    # Backpropagation
    loss.backward()     # backpropagates prediction loss, deposits gradients of the loss w.r.t. for each parameter that has requires_grad=True
    optimizer.step()    # updates the parameter values, "learns"
    optimizer.zero_grad()       # zeros the grads because it is a dynamic graph

  print(f"Epoch Accuracy: {matches/50000}")



In [9]:
def test(dataloader, model, device):

  # set module to testing mode, train does the opposite
  model.eval()
  accs = []
  matches = 0

  # used to turn off gradient calculations
  with torch.no_grad():
    for batch, (X,y) in enumerate(dataloader):
      imgs, labels = X.to(device), y.to(device)

      imgs = preprocess(imgs)
      pred = model(imgs)
      acc, m = accuracy(pred,torch.argmax(labels,dim=1))
      matches += m

      if batch % 50 == 0:
        print(f"Progress: {(batch+1)*len(X)}, current accuracy: {m/len(X)}")
        accs.append(acc)

  print(f"Test Accuracy: {matches/10000}")

In [10]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Pytorch version: {torch.__version__}")
    print(f"Device name: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("No GPU available.")

Pytorch version: 2.0.1+cu118
Device name: Tesla T4


In [18]:
model = vgg.VGG19().to(device)
model = nn.DataParallel(model)
# model.half() # If doing this, input type and weight type need to be the same

# set cross entropy loss and Adam optimizer functions
loss_fn = nn.CrossEntropyLoss()     # THIS NEEDS TARGET VALUES, NOT ONE-HOT ENCODING
optimizer = torch.optim.Adam(model.parameters()) # 5e-4 for my model

In [19]:
epochs = 5
start = datetime.now()
for e in range(epochs):
  train(train_loader, model, device, loss_fn, optimizer)
  test(test_loader,model,device)
elapse = datetime.now() - start
print("Training Time: ", elapse)


Progress: 64, current accuracy: 0.109375
Progress: 3264, current accuracy: 0.140625
Progress: 6464, current accuracy: 0.15625
Progress: 9664, current accuracy: 0.15625
Progress: 12864, current accuracy: 0.140625
Progress: 16064, current accuracy: 0.15625
Progress: 19264, current accuracy: 0.28125
Progress: 22464, current accuracy: 0.203125
Progress: 25664, current accuracy: 0.234375
Progress: 28864, current accuracy: 0.09375
Progress: 32064, current accuracy: 0.078125
Progress: 35264, current accuracy: 0.140625
Progress: 38464, current accuracy: 0.25
Progress: 41664, current accuracy: 0.203125
Progress: 44864, current accuracy: 0.265625
Progress: 48064, current accuracy: 0.234375
Epoch Accuracy: 0.17878
Progress: 64, current accuracy: 0.109375
Progress: 3264, current accuracy: 0.203125
Progress: 6464, current accuracy: 0.234375
Progress: 9664, current accuracy: 0.234375
Test Accuracy: 0.2152
Progress: 64, current accuracy: 0.171875
Progress: 3264, current accuracy: 0.265625
Progress: 6

In [12]:
# # MOVE THIS INTO THE FOR LOOP OR SMTH, like in the tutorial
# test(test_loader,model,device)

# Show missed images and what the machine thought it was
# Visualize the conv filter?